<a href="https://colab.research.google.com/github/mudogruer/SLMs/blob/main/phi-2/phi_2_fine_tuning_sciq_20pc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U accelerate bitsandbytes peft datasets trl

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.0 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.0.11 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which is incompatible.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.1.4 which is i

### This code involved Microsoft Phi model tine tuning with 20% of Sciq Dataset using with LoRA

In [ ]:
import transformers
transformers.__version__

'4.37.0'

In [ ]:
from transformers import AutoTokenizer

In [ ]:
base_model = "microsoft/phi-2"

In [ ]:
# from huggingface_hub import login
# login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    padding_side = "right",
    add_eos_token = True,
)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer

CodeGenTokenizerFast(name_or_path='microsoft/phi-2', vocab_size=50257, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	50257: AddedToken("                               ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50258: AddedToken("                              ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50259: AddedToken("                             ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	50260: AddedToken("                            ", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
	5

In [ ]:
import torch
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    load_in_4bit=True,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
model

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layern

In [ ]:
from datasets import load_dataset

dataset_name = "sciq"

In [ ]:
dataset = load_dataset(dataset_name)

In [ ]:
train_dataset = load_dataset(dataset_name, split ='train[:20%]')
eval_dataset = load_dataset(dataset_name, split='validation[:20%]')


In [ ]:
train_dataset.to_pandas().head(10)

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...
5,"When a meteoroid reaches earth, what is the re...",orbit,comet,meteor,meteorite,"Meteoroids are smaller than asteroids, ranging..."
6,What kind of a reaction occurs when a substanc...,nitrogen reaction,invention reaction,Fluid Reaction,combustion reaction,A combustion reaction occurs when a substance ...
7,Organisms categorized by what species descript...,species complex,surface species,fitting species,ring species,Ring species Ring species demonstrate a versio...
8,Alpha emission is a type of what?,light,radiation,heat,radioactivity,One type of radioactivity is alpha emission. W...
9,What is the stored food in a seed called?,larval,pollin,membrane,endosperm,The stored food in a seed is called endosperm ...


In [ ]:
train_dataset.to_pandas().dtypes

question          object
distractor3       object
distractor1       object
distractor2       object
correct_answer    object
support           object
dtype: object

In [ ]:
def generate_prompt(sample):
    full_prompt =f"""Instruct:{sample['question']}
    \nOutput:{sample['correct_answer']}"""
    return {"text": full_prompt}

In [ ]:
generate_prompt(train_dataset[1])

{'text': 'Instruct:What phenomenon makes global winds blow northeast to southwest or the reverse in the northern hemisphere and northwest to southeast or the reverse in the southern hemisphere?\n    \nOutput:coriolis effect'}

In [ ]:
generated_train_dataset = train_dataset.map(
    generate_prompt, remove_columns=list(train_dataset.features))
generated_val_dataset = eval_dataset.map(
    generate_prompt, remove_columns=list(eval_dataset.features))

Map:   0%|          | 0/2336 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
generated_train_dataset[5]["text"]

'Instruct:When a meteoroid reaches earth, what is the remaining object called?\n    \nOutput:meteorite'

In [ ]:
tokenizer(generated_train_dataset[5]["text"])

{'input_ids': [43993, 25, 2215, 257, 19999, 1868, 12229, 4534, 11, 644, 318, 262, 5637, 2134, 1444, 30, 198, 50284, 198, 26410, 25, 4164, 13492, 578], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, lora_config)

print_trainable_parameters(model)

trainable params: 4362240 || all params: 1525754880 || trainable%: 0.28590699968791844


In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bi

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_strategy="steps",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    max_steps=50,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    report_to="none",
)

In [ ]:
from trl import SFTTrainer

# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_arguments,
    train_dataset=generated_train_dataset,
    eval_dataset=generated_val_dataset,
    peft_config=lora_config,
    dataset_text_field="text",
)

2024-02-21 13:03:25.536239: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 13:03:25.536345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 13:03:25.693503: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:225: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2336 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
model.config.use_cache = False
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,2.969500,2.208048
50,2.087400,2.024803


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in

TrainOutput(global_step=50, training_loss=2.528489074707031, metrics={'train_runtime': 104.6019, 'train_samples_per_second': 1.912, 'train_steps_per_second': 0.478, 'total_flos': 102765603962880.0, 'train_loss': 2.528489074707031, 'epoch': 0.09})

In [ ]:
from huggingface_hub import login
login()

In [ ]:
my_finetuned_model = "Phi-2-hf-SciQ-20pc"

trainer.model.push_to_hub(my_finetuned_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mudogruer/Phi-2-hf-SciQ-20pc/commit/1bde94d2e2aae0d2526a2ce66051e7d4241fc647', commit_message='Upload model', commit_description='', oid='1bde94d2e2aae0d2526a2ce66051e7d4241fc647', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
model_phi =  AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    return_dict = True,
    load_in_4bit = True,
    device_map = "auto"
)
config = trainer.model.config

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
peft_model_id="mudogruer/Phi-2-hf-SciQ-20pc"

In [ ]:
from peft import PeftModel

fine_tuned_model = PeftModel.from_pretrained(
    model_phi,
    peft_model_id
)

adapter_config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

In [ ]:
fine_tuned_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bi

In [ ]:
from transformers import pipeline, logging

In [ ]:
logging.set_verbosity(logging.CRITICAL)

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = fine_tuned_model,
    tokenizer = tokenizer,
    eos_token_id = model.config.eos_token_id,
    max_new_tokens = 100,

)

In [ ]:
pipe = pipeline(
    task = "text-generation",
    model = fine_tuned_model,
    tokenizer = tokenizer,
    eos_token_id = model.config.eos_token_id,
    max_new_tokens = 30,

)

In [ ]:
prompt = "Which molecule in the air is the most?"
result = pipe(f"<s>Instruct:{prompt}/Output:")

In [ ]:
result

[{'generated_text': '<s>InstructWhich molecule in the air is the most?/Output:Oxygen\n    Output:Oxygen\n    Output:Oxygen\n    Output:Oxygen\n    Output:Oxy'}]

In [ ]:
prompt = "Which animal is the biggest?"
result = pipe(f"<s>Instruct:{prompt}/Output:")

In [ ]:
result

[{'generated_text': '<s>Instruct:Which animal is the biggest?/Output:Elephant</s>\n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n    \n'}]

In [ ]:
fused_model = "Sciq-Phi-20pc"

trainer.model.push_to_hub(fused_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mudogruer/Sciq-Phi-20pc/commit/385f12dba78f9aeec2fe9ff43a58c6f9d9c0e817', commit_message='Upload model', commit_description='', oid='385f12dba78f9aeec2fe9ff43a58c6f9d9c0e817', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
fine_tuned_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (model): PhiModel(
        (embed_tokens): Embedding(51200, 2560)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x PhiDecoderLayer(
            (self_attn): PhiAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2560, out_features=2560, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2560, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bi

In [ ]:
fine_tuned_model.push_to_hub(fused_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


adapter_model.safetensors:   0%|          | 0.00/279M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mudogruer/Sciq-Phi-20pc/commit/b064209b9e159435fafa8b91ee4ad828866102bb', commit_message='Upload model', commit_description='', oid='b064209b9e159435fafa8b91ee4ad828866102bb', pr_url=None, pr_revision=None, pr_num=None)